<p><font size="6"><b>Reshaping data</b></font></p>

---

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pivoting data

## Cfr. excel

Las personas que conocen Excel, probablemente conocen la funcionalidad **Pivot**:

![](img/pivot_excel.png)

Los datos de la tabla:

In [2]:
excelample = pd.DataFrame({'Month': ["January", "January", "January", "January",
                                  "February", "February", "February", "February",
                                  "March", "March", "March", "March"],
                   'Category': ["Transportation", "Grocery", "Household", "Entertainment",
                                "Transportation", "Grocery", "Household", "Entertainment",
                                "Transportation", "Grocery", "Household", "Entertainment"],
                   'Amount': [74., 235., 175., 100., 115., 240., 225., 125., 90., 260., 200., 120.]})

In [5]:
excelample

,Month,Category,Amount
0,January,Transportation,74.0
1,January,Grocery,235.0
2,January,Household,175.0
3,January,Entertainment,100.0
4,February,Transportation,115.0
5,February,Grocery,240.0
6,February,Household,225.0
7,February,Entertainment,125.0
8,March,Transportation,90.0
9,March,Grocery,260.0


In [4]:
# Create a spreadsheet-style pivot table as a DataFrame.
# The levels in the pivot table will be stored in MultiIndex objects
# (hierarchical indexes) on the index and columns of the result DataFrame.
excelample_pivot = excelample.pivot(index="Category", columns="Month", values="Amount")
excelample_pivot

Month,February,January,March
Category,,,
Entertainment,125.0,100.0,120.0
Grocery,240.0,235.0,260.0
Household,225.0,175.0,200.0
Transportation,115.0,74.0,90.0


¿Interesado en **totales totales**?

In [6]:
# sum columns
excelample_pivot.sum(axis=1)

Category
Entertainment     345.0
Grocery           735.0
Household         600.0
Transportation    279.0
dtype: float64

In [7]:
# sum rows
excelample_pivot.sum(axis=0)

Month
February    705.0
January     584.0
March       670.0
dtype: float64

## Pivot is just reordering your data

Pequeña submuestra del conjunto de datos de titanic:

In [8]:
df = pd.DataFrame({'Fare': [7.25, 71.2833, 51.8625, 30.0708, 7.8542, 13.0],
                   'Pclass': [3, 1, 1, 2, 3, 2],
                   'Sex': ['male', 'female', 'male', 'female', 'female', 'male'],
                   'Survived': [0, 1, 0, 1, 0, 1]})

In [9]:
df

,Fare,Pclass,Sex,Survived
0,7.2500,3,male,0
1,71.2833,1,female,1
2,51.8625,1,male,0
3,30.0708,2,female,1
4,7.8542,3,female,0
5,13.0000,2,male,1


In [10]:
df.pivot(index='Pclass', columns='Sex', values='Fare')

Sex,female,male
Pclass,,
1,71.2833,51.8625
2,30.0708,13.0000
3,7.8542,7.2500


In [11]:
df.pivot(index='Pclass', columns='Sex', values='Survived')

Sex,female,male
Pclass,,
1,1,0
2,1,1
3,0,0


So far, so good...

Ahora usemos el full titanic dataset:

In [12]:
df = pd.read_csv("data/titanic.csv")

In [13]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Intente el mismo pivote (**no se preocupe por el intento, excepto que aquí solo se usa para detectar un error muuuucho **):

In [14]:
try:
    df.pivot(index='Sex', columns='Pclass', values='Fare')
except Exception as e:
    print("Exception!", e)

Exception! Index contains duplicate entries, cannot reshape


Esto no funciona, porque terminaríamos con múltiples valores para una celda del marco resultante, como dice el error: valores `duplicated`" para las columnas de la selección. Como ejemplo, considere las siguientes filas de nuestras tres columnas de interés:


In [15]:
df.loc[[1, 3], ["Sex", 'Pclass', 'Fare']]

,Sex,Pclass,Fare
1,female,1,71.2833
3,female,1,53.1000


Dado que `pivot` es solo la reestructuración de datos, ¿dónde deberían ir los dos valores de` Fare` para la misma combinación de `Sex` y` Pclass`?

Bueno, deben combinarse, de acuerdo con una funcionalidad de `agregación`, que es compatible con la función`pivot_table`

<div class="alert alert-danger">

<b>NOTE</b>:

 <ul>
  <li> **Pivot** es puramente reestructuración: se requiere un valor único para cada combinación de índice / columna. </li>
</ul>
</div>

# Pivot tables - aggregating while pivoting

In [16]:
df = pd.read_csv("data/titanic.csv")

In [19]:
df.pivot_table(index='Sex', columns='Pclass', values='Fare')

Pclass,1,2,3
Sex,,,
female,106.125798,21.970121,16.118810
male,67.226127,19.741782,12.661633


<div class="alert alert-info">

<b>REMEMBER</b>:

 <ul>
  <li> De forma predeterminada, `pivot_table` toma la **media** de todos los valores que terminarían en una celda. Sin embargo, también puede especificar otras funciones de agregación usando la palabra clave `aggfunc`. </li>
</ul>
</div>

In [20]:
df.pivot_table(index='Sex', columns='Pclass',
               values='Fare', aggfunc='max')

Pclass,1,2,3
Sex,,,
female,512.3292,65.0,69.55
male,512.3292,73.5,69.55


In [ ]:
df.pivot_table(index='Sex', columns='Pclass',
               values='Fare', aggfunc='count')

<div class="alert alert-info">

<b>REMEMBER</b>:

 <ul>
  <li> Existe una función de acceso directo para una `tabla_dinámica` con un` aggfunc = count` como agregación: `crosstab` </li>
</ul>
</div>

In [ ]:
pd.crosstab(index=df['Sex'], columns=df['Pclass'])

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Haga una tabla dinámica con las tasas de supervivencia (= número de personas sobrevividas / número total de personas) para Pclass vs Sex. </li>
  <li> Trace el resultado como un gráfico de barras. </li>
</ul>
</div>

In [ ]:
df.pivot_table(index='Pclass', columns='Sex',
               values='Survived', aggfunc='mean')

In [ ]:
fig, ax1 = plt.subplots()
df.pivot_table(index='Pclass', columns='Sex',
               values='Survived', aggfunc='mean').plot(kind='bar',
                                                       rot=0,
                                                       ax=ax1)
ax1.set_ylabel('Survival ratio')

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Haga una tabla de la tarifa promedio pagada por edad / menor frente al sexo. </li>
</ul>
</div>

In [ ]:
df['Underaged'] = df['Age'] <= 18

In [ ]:
df.pivot_table(index='Underaged', columns='Sex',
               values='Fare', aggfunc='mean')

# Melt

La función `melt` realiza la operación inversa de un `pivot`. Esto se puede usar para alargar su marco, es decir, para hacer una versión *ordenada* de sus datos.

In [ ]:
pivoted = df.pivot_table(index='Sex', columns='Pclass', values='Fare').reset_index()
pivoted.columns.name = None

In [ ]:
pivoted

Supongamos que tenemos un DataFrame como el anterior. Las observaciones (la tarifa promedio pagada por las personas) se distribuyen en diferentes columnas. En un conjunto de datos ordenado, cada observación se almacena en una fila. Para obtener esto, podemos usar la función `melt`:

In [ ]:
pd.melt(pivoted)

Como puede ver arriba, la función `melt` coloca todas las etiquetas de columna en una columna y todos los valores en una segunda columna.

En este caso, esto no es completamente lo que queremos. Nos gustaría mantener la columna 'Sexo' por separado:

In [ ]:
pd.melt(pivoted, id_vars=['Sex']) #, var_name='Pclass', value_name='Fare')

# Reshaping with `stack` and `unstack`

La documentacion dice:

> Girar un nivel de las etiquetas de columna (posiblemente jerárquicas), devolviendo un
DataFrame (o Series en el caso de un objeto con un solo nivel de
etiquetas de columna) que tienen un índice jerárquico con un nuevo nivel más interno
de etiquetas de fila.

<img src="img/schema-stack.svg" width=50%>

Antes de hablar sobre el `hierarchical index`, primero compruébalo en la práctica en el siguiente ejemplo ficticio:

In [ ]:
df = pd.DataFrame({'A':['one', 'one', 'two', 'two'],
                   'B':['a', 'b', 'a', 'b'],
                   'C':range(4)})
df

Para usar `stack` /` unstack`, necesitamos los valores que queremos cambiar de filas a columnas o al revés como índice:

In [ ]:
df = df.set_index(['A', 'B']) # Indeed, you can combine two indices
df

In [ ]:
result = df['C'].unstack()
result

In [ ]:
df = result.stack().reset_index(name='C')
df

<div class="alert alert-info">

<b>REMEMBER</b>:

 <ul>
  <li> **pila **: haz que tus datos sean *más largos* y *más pequeños* </li>
  <li> **desapilar **: haz que tus datos sean *más cortos* y *más anchos* </li>
</ul>
</div>

## Mimick pivot table

Para comprender mejor y razonar sobre las tablas dinámicas, podemos expresar este método como una combinación de pasos más básicos. En resumen, el pivote es una forma conveniente de expresar la combinación de un `groupby` y` stack / unstack`.

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
df.head()

In [ ]:
df.pivot_table(index='Pclass', columns='Sex',
               values='Survived', aggfunc='mean')

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Obtenga el mismo resultado que el anterior basado en una combinación de `groupby` y` unstack` </li>
  <li> Primero use `groupby` para calcular la tasa de supervivencia para todos los grupos </li>
  <li> Luego, use `unstack` para remodelar la salida de la operación groupby </li>
</ul>
</div>

In [ ]:
df.groupby(['Pclass', 'Sex'])['Survived'].mean().unstack()

## Mimick melt

Al igual que la tabla dinámica anterior, ahora también podemos obtener el resultado de `melt` con apilar / desapilar.

Usemos el mismo marco `pivoted` que el anterior y observemos el resultado de fusión final:

In [ ]:
pivoted = df.pivot_table(index='Sex', columns='Pclass', values='Fare').reset_index()
pivoted.columns.name = None
pivoted

In [ ]:
pd.melt(pivoted, id_vars=['Sex'], var_name='Pclass', value_name='Fare')

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Obtenga el mismo resultado que el anterior usando `stack` /` unstack` (combinado con `set_index` /` reset_index`) </li>
  <li> Sugerencia: configure esas columnas como el índice que no desea apilar </li>
</ul>
</div>

In [ ]:
temp = pivoted.set_index('Sex')
temp

In [ ]:
temp.columns.name = 'Pclass'
temp = temp.stack()
temp

In [ ]:
temp.reset_index(name='Fare')

In [ ]:
# alternative: rename columns at the end
temp = pivoted.set_index('Sex').stack().reset_index()
temp.rename(columns={'level_1': 'Pclass', 0: 'Fare'})

# Exercises: use the reshaping methods with the movie data

These exercises are based on the ] [`titles.csv`]and [`cast.csv`] and put them in the `/data` folder.

In [ ]:
cast = pd.read_csv('data/cast.csv')
cast.head()

In [ ]:
titles = pd.read_csv('data/titles.csv')
titles.head()

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Trace la cantidad de papeles de actor cada año y la cantidad de papeles de actriz cada año durante todo el período de datos de películas disponibles. </li>
</ul>
</div>

In [ ]:
grouped = cast.groupby(['year', 'type']).size()
table = grouped.unstack('type')
table.plot()

In [ ]:
cast.pivot_table(index='year', columns='type', values="character", aggfunc='count').plot()
# for values in using the , take a column with no Nan values in order to count effectively all values -> at this stage: aha-erlebnis about crosstab function(!)

In [ ]:
pd.crosstab(index=cast['year'], columns=cast['type']).plot()

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Trace la cantidad de papeles de actor cada año y la cantidad de papeles de actriz cada año. Utilice kind = 'area' como tipo de parcela </li>
</ul>
</div>

In [ ]:
pd.crosstab(index=cast['year'], columns=cast['type']).plot(kind='area')

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li>Plot the fraction of roles that have been 'actor' roles each year over the whole period of available movie data.</li>
</ul>
</div>

In [ ]:
grouped = cast.groupby(['year', 'type']).size()
table = grouped.unstack('type')
(table['actor'] / (table['actor'] + table['actress'])).plot(ylim=[0,1])

<div class="alert alert-success">

<b>EXERCISE</b>:

 <ul>
  <li> Defina un año como un "año de Superman" cuando las películas de ese año presenten más personajes de Superman que de Batman. ¿Cuántos años en la historia del cine han sido los años de Superman? </li>
</ul>
</div>

In [ ]:
sel = cast[(cast.character == 'Superman') | (cast.character == 'Batman')]
sel = sel.groupby(['year', 'character']).size()
sel = sel.unstack()
sel = sel.fillna(0)
sel.head()

In [ ]:
d = sel['Superman'] - sel['Batman']
print('Superman years:')
print(len(d[d > 0.0]))